http://www.cse.chalmers.se/~richajo/dit866/assignments/a1/assignment1.html

We will particularly focus on decision tree learning for classification and regression.

Didactic purpose of this assignment:

getting a feel for the workflow of machine learning in Python;
understanding decision tree learning algorithms for classification and regression;
introducing the notions of overfitting and underfitting.

In [1]:
# task 0 
import sklearn
import pandas as pd
import numpy as np

print(sklearn.__version__)
print(pd.__version__)

1.1.2
1.2.5


# Task 1: A classification example: fetal heart condition diagnosis

In [3]:
## Task 1
import pandas as pd
from sklearn.model_selection import train_test_split
  
# Read the CSV file.
data = pd.read_csv("/work/CTG.csv", skiprows=1)

# Select the relevant numerical columns.
selected_cols = ['LB', 'AC', 'FM', 'UC', 'DL', 'DS', 'DP', 'ASTV', 'MSTV', 'ALTV',
                 'MLTV', 'Width', 'Min', 'Max', 'Nmax', 'Nzeros', 'Mode', 'Mean',
                 'Median', 'Variance', 'Tendency', 'NSP']
data = data[selected_cols].dropna()

# Shuffle the dataset.
data_shuffled = data.sample(frac=1.0, random_state=0)

# Split into input part X and output part Y.
X = data_shuffled.drop('NSP', axis=1)

# Map the diagnosis code to a human-readable label.
def to_label(y):
    return [None, 'normal', 'suspect', 'pathologic'][(int(y))]

Y = data_shuffled['NSP'].apply(to_label)

# Partition the data into training and test sets.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=0)




FileNotFoundError: [Errno 2] No such file or directory: '/CTG.csv'

In [3]:
# take a peek 
#X.head()

# dummyclasifier 
from sklearn.dummy import DummyClassifier
clf_dummy = DummyClassifier(strategy='most_frequent')

#get an idea of how well our simple classifier works
from sklearn.model_selection import cross_val_score
cross_val_score(clf_dummy, Xtrain, Ytrain)
#Results: array([0.78235294, 0.78235294, 0.77941176, 0.77941176, 0.77941176])

array([0.78235294, 0.78235294, 0.77941176, 0.77941176, 0.77941176])

In [4]:
#Tree-based classifiers 

#treeclassifier (no hyperparameter?)
from sklearn.tree import DecisionTreeClassifier
clf_tree = DecisionTreeClassifier(random_state=0) 
res = np.mean(cross_val_score(clf_tree, Xtrain, Ytrain))
print("Results from treeclassifier "  + str(round(res, 3)))

#Random forest classifier (depth of the forest)
from sklearn.ensemble import RandomForestClassifier
clf_random_forest = RandomForestClassifier(max_depth=100, random_state=1)
res2 = np.mean(cross_val_score(clf_random_forest, Xtrain, Ytrain))
print("Results from random forest classifier  " + str(round(res2, 3)))

Results from treeclassifier 0.924
Results from random forest classifier  0.944


In [5]:
#Linear models 

# Linear preceptron(tolerance strangly seems to have no impact)
from sklearn.linear_model import Perceptron
clf_perceptron = Perceptron(tol=1e-3, random_state=0)
res3 = np.mean(cross_val_score(clf_perceptron, Xtrain, Ytrain))
print("Results from Linear preceptron " + str(round(res3, 3)))
#Results: 
# -3 array([0.84705882, 0.88529412, 0.82941176, 0.85294118, 0.71176471])
# -1 array([0.84705882, 0.88529412, 0.82941176, 0.85294118, 0.71176471])
# -6 array([0.84705882, 0.88529412, 0.82941176, 0.85294118, 0.71176471])

# (linear) LogisticRegression 
from sklearn.linear_model import LogisticRegression
clf_log_reg = LogisticRegression(random_state=0,max_iter=1000, solver='newton-cg').fit(Xtrain, Ytrain)
# Had to swap lbfgs to newton-cg for it to work... 
res4 = np.mean(cross_val_score(clf_log_reg, Xtrain, Ytrain))
print("Results from LogisticRegression  " + str(round(res4, 3)))


Results from Linear preceptron 0.825
Results from LogisticRegression  0.891


In [6]:
#Linear SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
#from sklearn.datasets import make_classification
#X, y = make_classification(n_features=4, random_state=0)
clf_SVC = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5)) 
clf_SVC.fit(Xtrain, Ytrain)
# not converging properly

#Pipeline(steps=[('standardscaler', StandardScaler()),('linearsvc', LinearSVC(random_state=0, tol=1e-05))])
#res5 = np.mean(cross_val_score(clf_SVC, Xtrain, Ytrain))
#print("Results from SVC  " + str(round(res5, 3)))

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(random_state=0, tol=1e-05))])

In [ ]:
# Neural network classifier (will take longer time to train):
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
clf_MLPC = MLPClassifier(hidden_layer_sizes=(10,), random_state=1, max_iter=300) #.fit(Xtrain, Ytrain) 
res6 = np.mean(cross_val_score(clf_MLPC, Xtrain, Ytrain))
print("Results from Neural network classifier  " + str(round(res6, 3)))
# 100 layers give 0,856
# 1000 layers give 0,885

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
Results from Neural network classifier  0.881
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.

For the report. In your submitted report, please include a description of the classifier you selected and report its accuracy. (At this point, we are of course not asking you to describe internal workings of various machine learning models that we will cover in detail at later points during the course, but you are of course free to read about them if you're interested.)

In [8]:
## Step 4. Final evaluation
from sklearn.metrics import accuracy_score
clf_random_forest.fit(Xtrain, Ytrain)
Yguess = clf_random_forest.predict(Xtest)
print(accuracy_score(Ytest, Yguess))
# resulting score ≈0.93 from realier 0.94

0.9295774647887324


## Task #2 - in lecture1 NB 

## Task #3 - A regression example: predicting apartment prices

In [9]:
import numpy as np

# Read the CSV file using Pandas.
alldata = pd.read_csv("/work/sberbank.csv")

# Convert the timestamp string to an integer representing the year.
def get_year(timestamp):
    return int(timestamp[:4])
alldata['year'] = alldata.timestamp.apply(get_year)

# Select the 9 input columns and the output column.
selected_columns = ['price_doc', 'year', 'full_sq', 'life_sq', 'floor', 'num_room', 'kitch_sq', 'full_all']
alldata = alldata[selected_columns]
alldata = alldata.dropna()

# Shuffle.
alldata_shuffled = alldata.sample(frac=1.0, random_state=0)

# Separate the input and output columns.
X = alldata_shuffled.drop('price_doc', axis=1)
# For the output, we'll use the log of the sales price.
Y = alldata_shuffled['price_doc'].apply(np.log)

# Split into training and test sets.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=0)


We train a baseline dummy regressor (which always predicts the same value) and evaluate it in a cross-validation setting.

This example looks quite similar to the classification example above. The main differences are (a) that we are predicting numerical values, not symbolic values; (b) that we are evaluating using the mean squared error metric, not the accuracy metric that we used to evaluate the classifiers.

In [10]:
# creating baseline
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_validate
m1 = DummyRegressor()
rr= np.mean(cross_validate(m1, Xtrain, Ytrain, scoring='neg_mean_squared_error')['test_score'])
print("Resulting neg_mean_squared_error from crossvalidation, ie. baseline: " + str(round(rr, 3)))

Resulting neg_mean_squared_error from crossvalidation, ie. baseline: -0.389


#Testing different models

In [11]:
#sklearn.linear_model.Ridge
from sklearn.linear_model import Ridge
import numpy as np
rgr1 = Ridge(alpha=1)
#rgr1.fit(Xtrain, Ytrain)
res1 = np.mean(cross_val_score(rgr1, Xtrain, Ytrain))
print("Results from Ridge "  + str(round(res1, 3)))

Results from Ridge 0.224


In [1]:
from sklearn import linear_model, Lasso
rgr2 = Lasso(alpha=7)
#rgr2.fit(Xtrain, Ytrain)
res2 = np.mean(cross_val_score(rgr2, Xtrain, Ytrain))
print("Results from Lasso "  + str(round(res2, 3)))

ImportError: cannot import name 'Lasso' from 'sklearn' (/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/__init__.py)

In [36]:
#sklearn.tree.DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
rgr3 = DecisionTreeRegressor(criterion='absolute_error', max_depth=5,random_state=0)
res3 = np.mean(cross_val_score(rgr3, Xtrain, Ytrain, cv=10))
print("Results from DecisionTreeRegressor "  + str(round(res3, 3)))

Results from DecisionTreeRegressor 0.228


In [14]:
#sklearn.ensemble.RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
rgr4 = RandomForestRegressor(criterion='absolute_error', n_estimators=1,max_depth=1, random_state=0)
#rgr4.fit(Xtrain, Ytrain)
res4 = np.mean(cross_val_score(rgr4, Xtrain, Ytrain))
print("Results from RandomForestRegressor "  + str(round(res4, 3)))

Results from RandomForestRegressor 0.115


In [15]:
#sklearn.neural_network.MLPRegressor
from sklearn.neural_network import MLPRegressor
regr5 = MLPRegressor(hidden_layer_sizes=(100), random_state=1, max_iter=200) #.fit(Xtrain, Ytrain)
res5 = np.mean(cross_val_score(regr5, Xtrain, Ytrain))
print("Results from MLPRegressor "  + str(round(res5, 3)))

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:709: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:709: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:709: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
Results from MLPRegressor -243.887
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:709: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [39]:
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
rgr = linear_model.Lasso(alpha=0.01)
rgr.fit(Xtrain, Ytrain)
print(mean_squared_error(Ytest, rgr.predict(Xtest)))

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
rgr0 = DecisionTreeRegressor(criterion='absolute_error', max_depth=1,random_state=0)
rgr0.fit(Xtrain, Ytrain)
print(mean_squared_error(Ytest, rgr0.predict(Xtest)))

#sklearn.ensemble.RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
rgr9 = RandomForestRegressor(criterion='absolute_error', n_estimators=100, random_state=0)
rgr9.fit(Xtrain, Ytrain)
print(mean_squared_error(Ytest, rgr9.predict(Xtest)))

0.3180177945195446
0.3542452670890808
0.2998346331515957


Comments on the tuning. From the initial values before the tune model such as ridge, lasso and decision tree seemd most promising. However, we didn't know how big the improvements would be and hence tested to tune all models. After the tuning the best results are Results from Lasso -0.001, Results from DecisionTreeRegressor 0.115 (or a decision forest with only one tree). Ridge gave 0.224, and the neural network gave -50 which is by far the worst result.  

Comments on final evaluation: the best results with lasso were 0.31 when alfa is 0.1 as default value. Second best seems to be 

In [ ]:
# tuning the Lasso to right hyperparameter values
def a=0.001
for x in range(0, 5):
    LassoC = Lasso(alpha = a)
    print("Lasso with a=" +str(a)+": " +str(regression_cross_val_calc(cross_validate(LassoC, Xtrain, Ytrain, scoring='neg_mean_squared_error'))))
    a *=10 
# best/smallest obtained error is with a=0.1

In [ ]:
# tuning the DecisionTreeRegressor to right hyperparameter values 
#min_samples_leaf = 10
a=2
for x in range(0, 5):
    DTR = DecisionTreeRegressor(min_samples_split=a)
    print("DTR with depth=" +str(a)+": " +str(regression_cross_val_calc(cross_validate(DTR, Xtrain, Ytrain, scoring='neg_mean_squared_error'))))
    a += 50 
# max depth: little to no effect
# number of nodes in tree: min_samples_split ≈ 350 seem to yeild best result
# tree pruning is not avaibale for scikit


In [ ]:
# tuning the GradientBoostingRegressor to right hyperparameter values
a=2
for x in range(0, 5):
    GBR = GradientBoostingRegressor(min_samples_split=a)
    print("DTR with a=" +str(a)+": " +str(regression_cross_val_calc(cross_validate(GBR, Xtrain, Ytrain, scoring='neg_mean_squared_error'))))
    a += 10
# Loss: best results with'squared_error'
# n_estimators(The number of trees in the forest): good enough after 200+
# max_depth: around 5 seems to be sweet spot
# min_samples_split: no big effect
# tree pruning is not avaibale for scikit

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=727b6f02-a414-4cc8-9dec-71ed60bb1da5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>